<a href="https://colab.research.google.com/github/juwetta/DLI_Group-B/blob/main/DLI_Malicious_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Import imporant libraries

In [ ]:
import os
from sklearn.datasets import load_svmlight_file
import glob
import scipy.sparse
import numpy as np

Accessing to folder

In [ ]:
folder_path = '/content/drive/My Drive/DLI Group B/url_svmlight'

svm_files = glob.glob(os.path.join(folder_path, "*.svm"))
print(f"Found {len(svm_files)} SVM files in: {folder_path}")

Found 121 SVM files in: /content/drive/My Drive/DLI Group B/url_svmlight


In the context of the output, "shape" refers to the dimensions of the data structure (like a NumPy array or a SciPy sparse matrix).

For a 2D structure like combined_X, the shape (2396130, 3231961) means it has 2,396,130 rows and 3,231,961 columns. In this dataset, the rows represent the samples (e.g., URLs), and the columns represent the features.
For a 1D structure like combined_y, the shape (2396130,) means it has 2,396,130 elements. This corresponds to the labels for each of the samples in combined_X.
So, the shape tells you how many samples you have and how many features or labels are associated with each sample.

Read SVM file and store dataset


the different SVM files have varying numbers of features, causing an error when trying to combine them. I'll add a step to determine the total number of features across all files and then load each file with that consistent number of features.

In [ ]:
max_features = 0

for file_path in svm_files:
  try:
    X, _ = load_svmlight_file(file_path)
    if X.shape[1] > max_features:
      max_features = X.shape[1]

  except Exception as e:
    print(f"Error loading file {os.path.basename(file_path)}: {e}")


print(f"Maximum number of features found: {max_features}") #146.758s used

Maximum number of features found: 3231961


In [ ]:

all_X = []
all_y = []
max_features = 3231961
try:
    print("\nLoading and combining data...")

    # Load and combine data in a single pass, specifying the number of features
    for file_path in svm_files:
        try:
            X, y = load_svmlight_file(file_path, n_features=max_features)
            all_X.append(X)
            all_y.append(y)
            print(f"{os.path.basename(file_path)}", end="| ")

        except Exception as e:
            print(f"Error loading file {os.path.basename(file_path)}: {e}")

    if all_X and all_y:
        # Vertically stack the sparse feature matrices
        combined_X = scipy.sparse.vstack(all_X)
        # Concatenate the label arrays
        combined_y = np.concatenate(all_y)

        print("\nSuccessfully combined data from all files.")
        print(f"Shape of combined data (X): {combined_X.shape}")
        print(f"Shape of combined labels (y): {combined_y.shape}")
    else:
        print("\nNo data was loaded from the SVM files.")


except FileNotFoundError:
    print(f"Folder not found at: {folder_path}")
except Exception as e:
    print(f"An error occurred: {e}") #154.318s used


Loading and combining data...

Successfully combined data from all files.
Shape of combined data (X): (2396130, 3231961)
Shape of combined labels (y): (2396130,)


Identify the indexes of real-valued features

In [ ]:
import os

feature_types_path = '/content/drive/My Drive/DLI Group B/url_svmlight/FeatureTypes'
real_valued_feature_indices = set()

try:
    with open(feature_types_path, 'r') as f:
        for line in f:
            # Assuming each line in FeatureTypes is a feature index
            try:
                index = int(line.strip())
                real_valued_feature_indices.add(index)
            except ValueError:
                # Handle potential non-integer lines in the file
                print(f"Skipping non-integer line in FeatureTypes: {line.strip()}")

    print(f"Identified {len(real_valued_feature_indices)} real-valued feature indices.")
    # print("First 10 real-valued feature indices:", list(real_valued_feature_indices)[:10]) # Optional: print a few indices

except FileNotFoundError:
    print(f"FeatureTypes file not found at: {feature_types_path}")
except Exception as e:
    print(f"An error occurred while reading FeatureTypes: {e}")

# Now you have the set of real-valued feature indices and can use it
# For example, you could filter your data or analyze these specific features.

Identified 64 real-valued feature indices.


Briefly explore the dataset

In [ ]:
# Select the first few rows to inspect
num_rows_to_inspect = 5
sample_rows = combined_X[:num_rows_to_inspect]

print(f"Values of real-valued features in the first {num_rows_to_inspect} rows:")

# Iterate through the selected rows
for i in range(sample_rows.shape[0]):
    print(f"\nRow {i+1}:")
    # Iterate through the real-valued feature indices
    for feature_index in sorted(list(real_valued_feature_indices)): # Sorting for consistent output
        # Check if the feature exists in the current row (i.e., it's non-zero)

        if feature_index in sample_rows[i].indices:
            # Get the index within the non-zero elements
            data_index = np.where(sample_rows[i].indices == feature_index)[0][0]
            # Get the value of the feature
            feature_value = sample_rows[i].data[data_index]
            print(f"  Feature {feature_index}: {feature_value}")
        # If the feature index is not in sample_rows[i].indices, its value is 0 in the sparse matrix,
        # so we don't need to explicitly print 0 unless we want to see all real-valued features
        # even if their value is 0 for that sample. Let's only print non-zero real-valued features.

Values of real-valued features in the first 5 rows:

Row 1:
  Feature 4: 0.124138
  Feature 5: 0.117647
  Feature 16: 0.749633
  Feature 17: 0.843029
  Feature 18: 0.197344
  Feature 21: 0.142857
  Feature 22: 0.142857
  Feature 55: 1.0
  Feature 63: 1.0
  Feature 69: 1.0
  Feature 71: 1.0
  Feature 73: 1.0
  Feature 75: 1.0
  Feature 81: 1.0
  Feature 83: 1.0
  Feature 85: 1.0
  Feature 87: 1.0
  Feature 89: 1.0
  Feature 91: 1.0
  Feature 93: 1.0
  Feature 95: 1.0
  Feature 101: 1.0
  Feature 103: 1.0
  Feature 105: 1.0
  Feature 107: 1.0
  Feature 109: 1.0
  Feature 111: 1.0

Row 2:
  Feature 4: 0.103448
  Feature 5: 0.176471
  Feature 16: 0.72266
  Feature 17: 0.836498
  Feature 18: 0.6189
  Feature 21: 0.0119048
  Feature 23: 1.0
  Feature 55: 1.0
  Feature 61: 1.0
  Feature 63: 1.0
  Feature 65: 1.0
  Feature 67: 1.0
  Feature 69: 1.0
  Feature 71: 1.0
  Feature 73: 1.0
  Feature 75: 1.0
  Feature 81: 1.0
  Feature 83: 1.0
  Feature 85: 1.0
  Feature 87: 1.0
  Feature 89: 1.0
  F